In [140]:
import lyricsgenius
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
import requests
from urllib.request import urlopen
import bs4
from bs4 import BeautifulSoup
import re
import lxml.html
from lxml import html
from datetime import datetime, timedelta
import time
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import json
nltk.download('stopwords')
stop = stopwords.words('english')
sno = SnowballStemmer('english')
secrets = json.load(open('secrets.json','r'))

genius = lyricsgenius.Genius(secrets['genius_key'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anmar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [121]:


dom = lxml.html.fromstring(requests.get('https://genius.com/discussions/22873-Greatest-100-rappers').content)
rappers = [x for x in dom.xpath('//li/text()')]
rappers = [i for i in rappers if '\n' not in i]


base_url = "https://www.billboard.com/charts/artist-100/"
pop_big = []
for n in range(1):
    d = (datetime.now() - timedelta(days=60*n)).strftime('%Y-%m-%d')
    dom2 = lxml.html.fromstring(requests.get(base_url + d).content)
    pop = [x for x in dom2.xpath('//*[@class="chart-list-item__title-text"]/a/text()')]
    pop = [i[1:-1] for i in pop]
    pop_big += pop
    time.sleep(.1)
pop_big = list(set(pop_big))


dom3 = lxml.html.fromstring(requests.get('https://www.billboard.com/articles/news/dance/8504278/billboard-dance-100-2019').content)
electronic = [x for x in dom3.xpath('//p/strong/text()')]

no_words = ['Top Gig of 2018:', 'Outlook:', 'Notable Stat:','Contributors: ']
electronic=[i for i in electronic if not any(words in i for words in no_words)]


print(electronic)


['100. Bob Moses', '99. Lost Kings', '98. TOKiMONSTA', '97. Slushii', '96. Whethan', '95. Petit Biscuit', '94. Nina Kraviz', '93. CamelPhat', '92. Carnage', '91. Netsky', '90. Jax Jones', '89. The Martinez Brothers', '88. Felix Jaehn', '87. Showtek', '86. Lost Frequencies', '85. Deorro', '84. Marco Carola', '83. Jai Wolf', '82. NERVO', '81. Fisher', '80. Seven Lions', '79. MK', '78. Loud Luxury', '77. Nicky Romero', '76. Cash Cash', '75. Jonas Blue', '74. GRiZ', '73. Sofi\xa0Tukker', '72. 3LAU', '71. Alok', '70. SNAILS', '69. Madeon', '68. Maceo Plex', '67. Kayzo', '66. Big Gigantic', '65. KSHMR', '64. Richie Hawtin', '63. Oliver Heldens', '62. Adam Beyer', '61. ZHU', '60. Jauz', '59. Pretty Lights', '58. Matoma', '57. Black Coffee', '56. San Holo', '55. Gryffin', '54. Cashmere Cat', '53. Mura Masa', '52. W&W', '51. NGHTMRE', '50. Bonobo', '49. Robin Schulz', '48. Tchami', '47. R3hab', '46. Louis The Child', '45. Jamie Jones', '44. Cheat Codes', '43. RL Grime', '42. Yellow Claw', '41. 

In [122]:
electronic_fixed = []
for elec in electronic:
    electronic_fixed.append('. '.join(elec.split('. ')[1:]))
print(electronic_fixed)

['Bob Moses', 'Lost Kings', 'TOKiMONSTA', 'Slushii', 'Whethan', 'Petit Biscuit', 'Nina Kraviz', 'CamelPhat', 'Carnage', 'Netsky', 'Jax Jones', 'The Martinez Brothers', 'Felix Jaehn', 'Showtek', 'Lost Frequencies', 'Deorro', 'Marco Carola', 'Jai Wolf', 'NERVO', 'Fisher', 'Seven Lions', 'MK', 'Loud Luxury', 'Nicky Romero', 'Cash Cash', 'Jonas Blue', 'GRiZ', 'Sofi\xa0Tukker', '3LAU', 'Alok', 'SNAILS', 'Madeon', 'Maceo Plex', 'Kayzo', 'Big Gigantic', 'KSHMR', 'Richie Hawtin', 'Oliver Heldens', 'Adam Beyer', 'ZHU', 'Jauz', 'Pretty Lights', 'Matoma', 'Black Coffee', 'San Holo', 'Gryffin', 'Cashmere Cat', 'Mura Masa', 'W&W', 'NGHTMRE', 'Bonobo', 'Robin Schulz', 'Tchami', 'R3hab', 'Louis The Child', 'Jamie Jones', 'Cheat Codes', 'RL Grime', 'Yellow Claw', 'Solomun', 'Dillon Francis', 'Rezz', 'Don Diablo', 'Zeds Dead', 'Alison Wonderland', 'Illenium', 'Clean Bandit', 'Porter Robinson', 'Alan Walker', 'Afrojack', '', 'Disclosure', 'Carl Cox', 'Galantis', 'Eric Prydz', 'Kaskade', 'Justice', 'Bass

In [123]:
artists=rappers+pop_big+electronic_fixed

artists = list(set(artists))

print(len(artists))

281


In [124]:

artists = [art.replace('The Jungle Brothers', 'Jungle Brothers') for art in artists]
artists = [art.replace('The Funky Four+1', 'Funky Four Plus One') for art in artists]
artists = [art.replace('Kool Keith/Dr. Octagon', 'Kool Keith') for art in artists]

print(artists)
#print(len(artists))

# artists.append('Kool Keith')
#artists.index('Jungle Brothers')
#artists.index('Funky Four Plus One')
#artists.index('Kool Keith')



['Big L', 'Deorro', 'Jonas Blue', 'RL Grime', 'Lupe Fiasco', 'Kid ‘N Play', 'Thomas Rhett', 'N.W.A', 'Cheat Codes', 'Cypress Hill', 'Kayzo', 'Mobb Deep', '', 'Snoop Dogg', 'Skrillex', 'Afrika Bambaataa', 'Jungle Brothers', 'Taylor Swift', 'Gryffin', 'Lauren Daigle', 'Shawn Mendes', 'Felix Jaehn', 'Calvin Harris', 'Jean Grae', 'R.E.M.', 'Adam Beyer', 'Lil Kim', 'Dillon Francis', 'T.I.', 'R3hab', 'Tchami', 'Diplo', 'Justice', 'Lewis Capaldi', 'Queen Latifah', 'Public Enemy', 'Pete Rock & CL Smooth', 'Russell Dickerson', 'Notorious B.I.G.', 'Del the Funkee Homosapien', 'Nicky Romero', 'LL Cool J', 'Luke Bryan', 'Wu-Tang Clan', 'Dr. Dre', 'Eightball & MJG', 'DMX', 'Fisher', 'Jauz', 'Matoma', 'Black Coffee', 'W&W', 'Alan Walker', 'Axwell Ingrosso', 'Solomun', 'Schoolly D', 'Ghostface Killah', 'Geto Boys', 'Kane Brown', 'dead prez', 'Lost Frequencies', 'Gang Starr', 'The Pharcyde', 'Jon Pardi', 'The Martinez Brothers', 'Kurtis Blow', 'ZHU', 'Scarface', 'NF', 'Bob Dylan', 'Nas', 'Nirvana', 'D

In [125]:
lyric_dict = {}
for art in artists[0:2]:
    try:
        artist = genius.search_artist(art, max_songs=2,sort='popularity')
    except requests.exceptions.Timeout:
        continue
    if artist is None:
        continue
    for song in artist.songs:
        #print(song.artist + ' ' + song.title)
        lyric_dict[song.artist + '____' + song.title] = song.lyrics

lyric_df = pd.DataFrame.from_dict(lyric_dict,orient='index').reset_index()
lyric_df.columns = ['song_and_artist','lyrics']

def get_artist(word):
    return word.split('____')[0]

def get_song(word):
    return word.split('____')[1]

lyric_df['artist'] = lyric_df['song_and_artist'].apply(get_artist)
lyric_df['song'] = lyric_df['song_and_artist'].apply(get_song)
lyric_df.drop('song_and_artist',axis=1,inplace=True)

Searching for songs by Big L...

Changing artist name to 'Hayce Lemsi'
Song 1: "One-One"
Song 2: "Je t’aime moi non plus (#JTMNP)"

Reached user-specified song limit (2).
Done. Found 2 songs.
Searching for songs by Deorro...

Song 1: "Bailar"
Song 2: "Five More Hours"

Reached user-specified song limit (2).
Done. Found 2 songs.


In [126]:
lyric_df

,lyrics,artist,song
0,"[Couplet 1]\nJe recherche la forme et le fond,...",Hayce Lemsi,One-One
1,"[Intro]\nPutain, pourquoi elle répond pas ?\nO...",Hayce Lemsi,Je t’aime moi non plus (#JTMNP)
2,[Intro]\nDespacito\nMami yo sé que te va a gus...,Deorro,Bailar
3,"[Verse 1]\nWhat you wanna do, baby? Where you ...",Deorro,Five More Hours


In [127]:
lyric_df[lyric_df['song']=='One-One']

,lyrics,artist,song
0,"[Couplet 1]\nJe recherche la forme et le fond,...",Hayce Lemsi,One-One


In [128]:
def clean_html(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext

def clean_punc(word):
    cleaned = re.sub(r'[?|!|\'|#]', r'', word)
    cleaned = re.sub(r'[.|,|)|(|\|/]', r' ', cleaned)
    return cleaned

def text_preprocess(sentence):
    filtered_sentence = []
    for word in sentence.split():
        if word not in stop:
            s = sno.stem(word.lower())
            filtered_sentence.append(s)
        else:
            continue 
    return ' '.join(filtered_sentence)

lyric_df['lyrics']  = lyric_df['lyrics'].apply(clean_punc)
lyric_df['lyrics']  = lyric_df['lyrics'].apply(clean_html)
lyric_df['Cleaned Text']  = lyric_df['lyrics'].apply(text_preprocess)



# for sentence in lyric_df['lyrics'].values:
#     filtered_sentence = []
#     sentence = clean_html(sentence)
#     for word in sentence.split():
#         for cleaned_word in clean_punc(word).split():
#             if (cleaned_word.isalpha() and (len(cleaned_word) > 2) and cleaned_word not in stop):
#                 s = (sno.stem(cleaned_word.lower())).encode('utf8')
#                 filtered_sentence.append(s)
#             else:
#                 continue
                 
#     strl = b' '.join(filtered_sentence)
#     final_string.append(strl)

# lyric_df['Cleaned Text'] = final_string

In [129]:
lyric_df

,lyrics,artist,song,Cleaned Text
0,[Couplet 1]\nJe recherche la forme et le fond ...,Hayce Lemsi,One-One,[couplet 1] je recherch la form et le fond job...
1,[Intro]\nPutain pourquoi elle répond pas \nOu...,Hayce Lemsi,Je t’aime moi non plus (#JTMNP),[intro] putain pourquoi ell répond pas ouai ce...
2,[Intro]\nDespacito\nMami yo sé que te va a gus...,Deorro,Bailar,[intro] despacito mami yo sé que te va gustar ...
3,[Verse 1]\nWhat you wanna do baby Where you w...,Deorro,Five More Hours,[vers 1] what wanna babi where wanna go ill ta...


In [130]:
cv = CountVectorizer()
df = cv.fit_transform(lyric_df['Cleaned Text'])

count_vect_df_clean = pd.DataFrame(df.todense(), columns=['vec_' + i for i in cv.get_feature_names()])
lyric_df = lyric_df.join(count_vect_df_clean)

lyric_df

,lyrics,artist,song,Cleaned Text,vec_abajo,vec_allant,vec_alliag,vec_altess,vec_alway,vec_anoth,...,vec_zipett,vec_âge,vec_ça,vec_écout,vec_écrit,vec_éloquent,vec_érigé,vec_était,vec_êtr,vec_île
0,[Couplet 1]\nJe recherche la forme et le fond ...,Hayce Lemsi,One-One,[couplet 1] je recherch la form et le fond job...,0,0,0,0,0,0,...,1,1,2,1,0,1,0,0,0,0
1,[Intro]\nPutain pourquoi elle répond pas \nOu...,Hayce Lemsi,Je t’aime moi non plus (#JTMNP),[intro] putain pourquoi ell répond pas ouai ce...,0,1,1,2,0,0,...,0,0,0,0,1,0,4,2,2,1
2,[Intro]\nDespacito\nMami yo sé que te va a gus...,Deorro,Bailar,[intro] despacito mami yo sé que te va gustar ...,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,[Verse 1]\nWhat you wanna do baby Where you w...,Deorro,Five More Hours,[vers 1] what wanna babi where wanna go ill ta...,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [131]:
music_df = lyric_df

#print(artist_df)
#print(music_df)

In [ ]:
#music_df.to_csv("music.csv", index=False)


In [132]:
averaged_df = music_df.drop(['song','lyrics'],axis=1).groupby('artist').mean().reset_index()



In [133]:
from sklearn.metrics.pairwise import cosine_similarity
query='Weezer'
match = averaged_df[averaged_df['artist']==query]
rest_of_entries = averaged_df[averaged_df['artist']!=query].copy()

if match.empty:\
    print('Not Found!')
    #Throw something here
    #return None
print(cosine_similarity(match.loc[:, match.columns != 'artist'],rest_of_entries.loc[:,rest_of_entries.columns!='artist']))
rest_of_entries['similarity'] = cosine_similarity(match.loc[:, match.columns != 'artist'],rest_of_entries.loc[:,rest_of_entries.columns!='artist'])[0]
rest_of_entries

Not Found!


ValueError: Found array with 0 sample(s) (shape=(0, 629)) while a minimum of 1 is required by check_pairwise_arrays.

In [140]:
cosine_similarity(averaged_df.drop('artist',axis=1),averaged_df.drop('artist',axis=1))

array([[1.        , 0.82064323, 0.70700936, ..., 0.74918614, 0.69364512,
        0.6932935 ],
       [0.82064323, 1.        , 0.76728803, ..., 0.74985765, 0.69923893,
        0.67478045],
       [0.70700936, 0.76728803, 1.        , ..., 0.61287631, 0.54991866,
        0.56922461],
       ...,
       [0.74918614, 0.74985765, 0.61287631, ..., 1.        , 0.79791777,
        0.74379076],
       [0.69364512, 0.69923893, 0.54991866, ..., 0.79791777, 1.        ,
        0.7816415 ],
       [0.6932935 , 0.67478045, 0.56922461, ..., 0.74379076, 0.7816415 ,
        1.        ]])

In [134]:
averaged_df

,artist,vec_abajo,vec_allant,vec_alliag,vec_altess,vec_alway,vec_anoth,vec_apart,vec_aprieta,vec_aprè,...,vec_zipett,vec_âge,vec_ça,vec_écout,vec_écrit,vec_éloquent,vec_érigé,vec_était,vec_êtr,vec_île
0,Deorro,0.5,0.0,0.0,0.0,0.5,0.5,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Hayce Lemsi,0.0,0.5,0.5,1.0,0.0,0.0,0.0,0.0,0.5,...,0.5,0.5,1.0,0.5,0.5,0.5,2.0,1.0,1.0,0.5


In [135]:
def pull_genre(row):
    artist=row['artist'].replace(' ', '%20')
    CLIENT_ID = secrets['spotify_id']
    CLIENT_SECRET = secrets['spotify_secret']

    grant_type = 'client_credentials'
    body_params = {'grant_type' : grant_type}

    url='https://accounts.spotify.com/api/token'
    response = requests.post(url, data=body_params, auth = (CLIENT_ID, CLIENT_SECRET)) 

    token_raw = json.loads(response.text)
    token = token_raw["access_token"]
    headers = {"Authorization": "Bearer {}".format(token)}
    try:
        r = requests.get(url="https://api.spotify.com/v1/search?q={}&type=artist".format(artist), headers=headers)
        ret = json.loads(r.text)
        return ret['artists']['items'][0]['genres']
    except:
        return []

In [136]:
averaged_df['Genre'] = averaged_df.apply(pull_genre,axis=1)



In [137]:
averaged_df

,artist,vec_abajo,vec_allant,vec_alliag,vec_altess,vec_alway,vec_anoth,vec_apart,vec_aprieta,vec_aprè,...,vec_âge,vec_ça,vec_écout,vec_écrit,vec_éloquent,vec_érigé,vec_était,vec_êtr,vec_île,Genre
0,Deorro,0.5,0.0,0.0,0.0,0.5,0.5,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[big room, dance pop, deep big room, disco hou..."
1,Hayce Lemsi,0.0,0.5,0.5,1.0,0.0,0.0,0.0,0.0,0.5,...,0.5,1.0,0.5,0.5,0.5,2.0,1.0,1.0,0.5,"[francoton, french hip hop, pop urbaine]"


In [89]:
#averaged_df.to_csv("artist.csv", index=False)
#averaged_df = pd.read_csv("artist.csv")
